In [1]:
# 1. Import Libraries
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client
import numpy as np
import re



import torch
import torch.nn as nn
import torch.distributed as dist
from torch.utils.data import DataLoader, DistributedSampler, TensorDataset
from torch.optim import Adam
from torch.multiprocessing import Process
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
client = Client(n_workers=8, threads_per_worker=1, memory_limit='8GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 8,Total memory: 59.60 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58712,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 59.60 GiB
Comm: tcp://127.0.0.1:58745,Total threads: 1
Dashboard: http://127.0.0.1:58749/status,Memory: 7.45 GiB
Nanny: tcp://127.0.0.1:58715,


In [3]:
train_path = '../dataset/train/train.ft.txt'
test_path = '../dataset/test/test.ft.txt'

In [4]:
train_data_dask = dd.read_csv(train_path, delimiter='\t', header=None, dtype=str)
test_data_dask = dd.read_csv(test_path, delimiter='\t', header=None, dtype=str)

In [5]:
def process_data_dask(df):
  labels = df[0].str.split(' ', n=1, expand=True)[0].str.replace('__label__', '')
  text = df[0].str.split(' ', n=1, expand=True)[1]

  processed_df = pd.DataFrame({
    'label': labels,
    'review': text.str.strip()
  })
  return processed_df

In [6]:
meta = pd.DataFrame(columns=['label', 'review'], dtype=str)

In [7]:
train_data_dask = dd.read_csv(train_path, delimiter='\t', header=None, dtype=str)
test_data_dask = dd.read_csv(test_path, delimiter='\t', header=None, dtype=str)

In [8]:
train_data_dask = train_data_dask.map_partitions(process_data_dask, meta=meta)
test_data_dask = test_data_dask.map_partitions(process_data_dask, meta=meta)

In [9]:
test_data_dask = test_data_dask.repartition(npartitions=4)

In [10]:
train_data_dask = train_data_dask.persist()
test_data_dask = test_data_dask.persist()

In [11]:
replace_dict = {"2": "1", "1": "0"}

In [12]:
def replace_labels(df):
  df['label'] = df['label'].replace(replace_dict)
  return df

In [13]:
train_data_dask = train_data_dask.map_partitions(replace_labels)
test_data_dask = test_data_dask.map_partitions(replace_labels)

In [14]:
def text_cleaning(text):
  text = text.lower()
  pattern_punc = r'[^A-Za-z\s]'
  text = re.sub(pattern_punc, '', text).strip()
  return text

In [15]:
def clean_reviews(df):
  df['review'] = df['review'].apply(text_cleaning)
  df = df.dropna()
  return df

In [16]:
train_data_dask = train_data_dask.map_partitions(clean_reviews)
test_data_dask = test_data_dask.map_partitions(clean_reviews)

In [17]:
train_data_processed = train_data_dask.compute()
test_data_processed = test_data_dask.compute()

In [18]:
test_data_dask

,label,review
npartitions=4,,
,object,object
,...,...
,...,...
,...,...
,...,...


In [19]:
client.close()